In [ ]:
# -- IMPORTS --
'''
%pip install livelossplot
%pip install torcheval
%pip install torchmetrics
'''
import os
os.chdir('/workplace/flowmind/')
print("working in: " + os.getcwd())

#%run /workplace/flowmind/setup.py

import csv
import sys

# torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as FM
import torchvision.transforms as T
from torcheval.metrics import MulticlassAccuracy

# flowmind imports
#from flowmind.processing.dataloaders.flowpic import create_flowpic_dataloader 
#from flowmind.datasets.mirage import remap_label_mirage19
from flowmind.processing.dataloaders.common import FlowData

# livelossplot imports
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

# others
from datetime import timedelta
import random

os.chdir('/workplace/xcocek00/')
print("working in: " + os.getcwd())
from flowpic import create_flowpic_dataloader

csv.field_size_limit(sys.maxsize)

# sets the device to use gpu if available, if not, use cpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# -- RTT augmentation --

def augment_rtt(flow: FlowData, alpha_min: float = 0.5, alpha_max: float = 10.5) -> FlowData:
    """
    Multiply arrival time of each packet by a factor alpha, where
    alpha is chosen uniformly in [alpha_min, alpha_max]

    Args
        flow: original flow
        alpha_min: min factor set to 0.5
        alpha_max: max factor set to 1.5

    Returns: 
        modified FlowData
    """
    
    if not flow.times:
        return flow

    # select random alpha 
    alpha = random.uniform(alpha_min, alpha_max)
    init_time = flow.init_time

    # convert time to float offset
    offsets = [(t - init_time).total_seconds() for t in flow.times]
    
    # RTT augmentation
    offsets = [offset * alpha for offset in offsets]

    # convert back
    flow.times = [init_time + timedelta(seconds=offset) for offset in offsets]

    return flow


In [ ]:
# -- IAT augmentation --

def augment_iat(flow: FlowData, b_min: float = -1.0, b_max: float = 1.0) -> FlowData:
    '''
    Add factor b to the arrival time of each packet, where
    b is chosen uniformly in [b_min, b_max]

    Args:
        flow: original flow
        b_min: min factor set to -1.0 
        b_max: max factor set to 1.0

    Returns: 
        modified FlowData
    '''
    
    if not flow.times:
        return flow

    # select random b 
    b = random.uniform(b_min, b_max)
    init_time = flow.init_time

    # convert time to float offset
    offsets = [(t - init_time).total_seconds() for t in flow.times]
    
    # IAT augmentation
    offsets = [offset + b for offset in offsets]

    # convert back
    flow.times = [init_time + timedelta(seconds=offset) for offset in offsets]

    return flow

In [ ]:
# -- packet loss augmentation --

def packet_loss(flow: FlowData, dt: float = 0.1) -> FlowData:
    '''
    Packet loss augmentation - removing all packets within [t - dt, t + dt]
    
    Args:
        flow: original flow
        dt: delta t (interval length) set to 0.1
    
    Returns: 
        modified Flow with removed packets in that interval
    ''' 

    if not flow.times:
        return flow
    
    init_time = flow.init_time 
    
    # conver to offset
    offsets = [(t - init_time).total_seconds() for t in flow.times]
    flow_duration = offsets[-1] if offsets else 0.0

    # zero or near-zero duration -> skip (would remove the whole flow)
    # TODO: consider 2*dt
    if flow_duration <= 0:
        return flow

 
    t = random.uniform(0, flow_duration)

    # interval [t - dt, t + dt]
    lower = t - dt
    upper = t + dt

    # lists for filtered packets
    new_directions = []
    new_lengths    = []
    new_times      = []
    new_push_flags = []

    for offset, direction, length, time_val, push_flag in zip(offsets, flow.directions, flow.lengths, flow.times, flow.push_flags):
        if not (lower <= offset <= upper):
            new_directions.append(direction)
            new_lengths.append(length)
            new_times.append(time_val)
            new_push_flags.append(push_flag)

    # replace old lists with the fitered lists
    flow.directions = new_directions
    flow.lengths    = new_lengths
    flow.times      = new_times
    flow.push_flags = new_push_flags
    flow.init_time  = flow.times[0] if flow.times else None

    return flow


In [ ]:
# -- TESTS --

# dimensions validation
def dim_val(dl_train) -> None:
    print("dl train: ", type(dl_train))
    for (flowpic1, flowpic2, labels) in dl_train:
        print("flowpic1", type(flowpic1))
        print("flowpic2", type(flowpic2))
        print("flowpic1 shape: ", flowpic1.shape)
        print("flowpic2 shape: ", flowpic2.shape)
        break    

# print batch
def print_batch(dl_train) -> None:
    for batch in dl_train:
        torch.set_printoptions(threshold=sys.maxsize)
        print(batch)
        break
    
# debug batch    
def debug_batch(dl):
    for (flowpic1, flowpic2, labels) in dl:
        print("flowpic1 type:", type(flowpic1))
        print("flowpic2 type:", type(flowpic2))
        print("label type:", type(labels))
        
        if isinstance(flowpic1, list):
            print(f"flowpic1 has {len(flowpic1)} items.")
            print("Shapes of flowpic1 items:")
            for i, fp in enumerate(flowpic1):
                print(type(fp))
                print(f"  Item {i}: {fp.shape}")
                break

        if isinstance(flowpic2, list):
            print(f"flowpic2 has {len(flowpic2)} items.")
            print("Shapes of flowpic2 items:")
            for i, fp in enumerate(flowpic2):
                print(f"  Item {i}: {fp.shape}")

        print("labels:", labels)
        break  # Just inspect the first batch

In [ ]:
# -- custom remap_label function --
from functools import partial

def remap_label(t: tuple[torch.Tensor, torch.Tensor, str], labels_d: dict[str, int]) -> tuple[torch.Tensor, torch.Tensor, int]:
    return t[0], t[1], labels_d[t[2]]

labels_mirage19 = [
    "com.joelapenna.foursquared",
    "com.contextlogic.wish",
    "com.pinterest",
    "com.tripadvisor.tripadvisor",
    "com.groupon",
    "com.accuweather.android",
    "com.waze",
    "com.duolingo",
    "com.viber.voip",
    "com.facebook.katana",
    "de.motain.iliga",
    "it.subito",
    "com.facebook.orca",
    "com.dropbox.android",
    "com.twitter.android",
    "com.google.android.youtube",
    "com.spotify.music",
    "com.iconology.comics",
    "com.trello",
    "air.com.hypah.io.slither",
]


labels_mirage22 = [
    "com.Slack",
    "com.cisco.webex.meetings",
    "com.discord",
    "com.facebook.orca",
    "com.google.android.apps.meetings",
    "com.gotomeeting",
    "com.microsoft.teams",
    "com.skype.raider",
    "us.zoom.videomeetings",
]


labels_mirage19_d = {label: i for i, label in enumerate(labels_mirage19)}
labels_mirage22_d = {label: i for i, label in enumerate(labels_mirage22)}

remap_label_mirage19 = partial(remap_label, labels_d=labels_mirage19_d)
remap_label_mirage22 = partial(remap_label, labels_d=labels_mirage22_d)



In [ ]:
# -- DATA LOADERS --
# TODO: vyresit vyber augmentaci

# 0 - debug prints are turned off
# 1 - debug prints are turned on
DEBUG = 0

# dataloader yields tuple (flowpic, flowpic, label)
dl_train = create_flowpic_dataloader(
    #dir_path="/workplace/xcocek00/test_b.csv",
    dir_path="/workplace/datasets/mirage19/processed/splits/train.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (300 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    flow_transform_1=augment_rtt,
    flow_transform_2=augment_rtt,
    bidirectional = False,
    dp_transform=lambda dp: dp.map(remap_label_mirage19).in_memory_cache(),
)
dl_val = create_flowpic_dataloader(
    dir_path="/workplace/datasets/mirage19/processed/splits/val.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (300 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    flow_transform_1=augment_rtt,
    flow_transform_2=augment_rtt,
    bidirectional = False,
    dp_transform=lambda dp: dp.map(remap_label_mirage19).in_memory_cache(),
)

if DEBUG:    
    #print(type(dl_train))
    dim_val(dl_train)
    #print_batch(dl_train)
    #debug_batch(dl_train)

In [ ]:
# -- CNN architecture --

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # conv + pooling layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        # fully connected (fc) layers
        self.fc1 = nn.Linear(400, 120) # 16*5*5 = 400 
        self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120, 30)

    def forward(self, x):
        # conv + ReLU + pool 1
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        # conv + ReLU + pool 2
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # flatten for fc layers
        x = x.view(x.size(0), -1)
        
        # end of encoder f(·) -> h
        h = F.relu(self.fc1(x))
        
        # projection head g(·) -> z
        z = F.relu(self.fc2(h))   
        z = self.fc3(z)
        
        # L2 normalize
        z = F.normalize(z, dim=1)
        
        '''
        Returns:
          h: 120-d representation
          z: 30-d projection'
        '''
        return h, z
    

In [ ]:
# -- fine tuning using linear classifier --

class MLP(nn.Module):
    def __init__(self, output_size: int, input_size=120):
        super().__init__()
        self.input_size = input_size

        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        y = self.fc(x)
        return y


In [ ]:
# -- training loops--
# FIXME: delete custom acc

def train(model, dataloader, optimizer, contrastive_loss_fn, log):
    '''
    Training loop

    Args:
        model: CNN model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        optimizer: chosen optimizer
        contrastive_loss_fn: contrastive loss function (NT-Xent)
        log: for log collection

    Returns:
        Avarage loss 
    '''

    model.train()
    total_loss = 0.0
    batches = 0
    acc_total = 0.0

    # label not needed in -> _
    for flowpic1, flowpic2, _ in dataloader:
        flowpic1, flowpic2 = flowpic1.to(device), flowpic2.to(device)

        optimizer.zero_grad()

        # forward pass to get embeddings
        _, z1 = model(flowpic1)  
        _, z2 = model(flowpic2)  

        # contrastive loss
        loss = contrastive_loss_fn(z1, z2)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batches += 1
        
        batch_acc = contrastive_accuracy(z1, z2, temperature=0.07)
        acc_total += batch_acc.item()

    avg_loss = total_loss / batches
    avg_acc = acc_total / batches
    log["loss"] = avg_loss
    #log["accuracy"] = avg_acc

    return avg_loss

def val(model, dataloader, contrastive_loss_fn, log):
    '''
    Validation loop

    Args:
        model: CNN model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        contrastive_loss_fn: contrastive loss function (NT-Xent)
        log: for log collection

    Returns:
        Avarage loss 
    '''
    with torch.no_grad():
        model.eval()
        total_loss = 0.0
        batches = 0
        acc_total = 0.0

        # label not needed in -> _
        for flowpic1, flowpic2, _ in dataloader:
            flowpic1, flowpic2 = flowpic1.to(device), flowpic2.to(device)

            # forward pass to get embeddings
            _, z1 = model(flowpic1)  
            _, z2 = model(flowpic2)  

            # contrastive loss
            loss = contrastive_loss_fn(z1, z2)

            total_loss += loss.item()
            batches += 1
            
            batch_acc = contrastive_accuracy(z1, z2, temperature=0.07)
            acc_total += batch_acc.item()

        avg_loss = total_loss / batches
        avg_acc = acc_total / batches
        log["val_loss"] = avg_loss
        #log["val_accuracy"] = avg_acc

    return avg_loss

def classification(CNN_model, MLP_model, dataloader, log, loss_fn, accuracy, optimizer):
    '''
    Classification loop

    Args:
        CNN_model: CNN model 
        MLP_model: MLP model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        log: for log collection
        loss_fn: chosen loss function
        accuracy: chosen model accuracy
        optimizer: chosen optimizer

    Returns:
        classification loss 
    '''

    # freeze the encoder
    for param in CNN_model.parameters():
        param.requires_grad = False

    # set models
    CNN_model.eval()
    MLP_model.train()

    classification_loss = 0.0
    batches = 0

    for flowpic1, flowpic2, label in dataloader:
        optimizer.zero_grad()

        flowpic1, flowpic2, label = flowpic1.to(device), flowpic2.to(device), label.to(device)
        
        # forward pass to get embeddings
        with torch.no_grad():
            h, _ = CNN_model(flowpic1)  
            _, _ = CNN_model(flowpic2)  
        
        # use MLP for fine tuning
        y = MLP_model(h)
        
        loss = loss_fn(y, label)
        loss.backward()
        optimizer.step()

        classification_loss += loss.item()
        batches += 1
        accuracy.update(y, label)
    
    classification_loss /= batches
        
    log["classification_loss"] = classification_loss
    log["accuracy"] = accuracy.compute()
        
    accuracy.reset()

    return classification_loss



In [ ]:
# -- loss function --

def nt_xent_loss(z1: torch.Tensor, z2: torch.Tensor, temperature: float = 0.07) -> torch.Tensor:
    '''
    NT-Xent (Normalized Temperature-Scaled Cross-Entropy) loss function
    
    Args:
        z1: tensor 
        z2: tensor 
        temperature: scaling factor
    
    Returns:
        A scalar loss value
    '''
    
    batch_size = z1.shape[0]
    
    # concatenate embeddings -> shape [2N, embed_dim]
    z = torch.cat([z1, z2], dim=0)
    
    # cosine similarity matrix scaled by temperature
    sim_matrix = torch.matmul(z, z.T) / temperature
    
    # mask to remove self-similarities (diagonal entries)
    diag_mask = torch.eye(2 * batch_size, dtype=torch.bool, device=z.device)
    sim_matrix.masked_fill_(diag_mask, -float('inf'))
    
    # for each i the positive example is:
    # - i in [0, N-1] then positive is: i + N
    # - i in [N, 2N-1] then positive is: i - N
    positives = torch.cat([
        torch.arange(batch_size, 2 * batch_size),
        torch.arange(0, batch_size)
    ], dim=0).to(z.device)
    
    # cross entropy loss
    loss = F.cross_entropy(sim_matrix, positives)
    
    return loss

In [ ]:
# TODO: add more datasets
mirage19_classes = 20

# CNN definition
cnn_model = CNN()
cnn_model = cnn_model.to(device)

# optimizer
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

# NT-Xent loss function
contrastive_loss_fn = nt_xent_loss

cnn_liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6, 2))])

# Linear classifier definition
mlp_model = MLP(mirage19_classes)
mlp_model = mlp_model.to(device)

# optimizer
mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.01)

# loss function
mlp_loss_fn = nn.CrossEntropyLoss()

mlp_accuracy = MulticlassAccuracy()

mlp_liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6, 2))])

In [ ]:
# -- traning CNN --

for epoch in range(10):
    log = {}
    train_loss = train(cnn_model, dl_train, cnn_optimizer, contrastive_loss_fn, log)
    val_loss = val(cnn_model, dl_val, contrastive_loss_fn, log)

    cnn_liveloss.update(log)
    cnn_liveloss.send()

In [ ]:
# -- training linear classifier --
# TODO: add test set

for epoch in range(5):
    log = {}
    classification_loss = classification(cnn_model, mlp_model, dl_train, log, mlp_loss_fn, mlp_accuracy, mlp_optimizer)

    mlp_liveloss.update(log)
    mlp_liveloss.send()